In [3]:
import os
import pandas as pd
import ast
import re

# create the mapping dictionary and Choose residental and non_residental

In [2]:
# xml = http://www.sig3d.org/codelists/Handbuch-SIG3D/building/2.0/CL-V1.0/_AbstractBuilding_function.xml
with open(r"D:\Nicola project\Nicola project\1\database\preprocessing\1_parsing\Germen\dictionary.txt",'r',encoding='utf-8') as f:
    dictionary =f.read()
    


In [3]:
map_dictionary = pd.DataFrame()
map_dictionary['type'] = re.findall(r"<gml:description>(.+?)</gml:description>\n", dictionary)
map_dictionary['code']= re.findall(r"<gml:name>(.+?)</gml:name>\n", dictionary)

In [4]:
# pip install deep-translator
from deep_translator import GoogleTranslator

In [5]:
map_dictionary['german_name'] = map_dictionary['type']

In [6]:
lst = []
for i in map_dictionary['type']:
    name = GoogleTranslator(source='auto', target='en').translate(i)
    lst.append(name)
    


In [7]:
map_dictionary['english_name'] = lst

In [8]:
map_dictionary

,type,code,german_name,english_name
0,Wohngebäude,1000,Wohngebäude,residential building
1,Wohnhaus,1010,Wohnhaus,residential house
2,Wohnheim,1020,Wohnheim,dorm
3,Kinderheim,1021,Kinderheim,children's home
4,Seniorenheim,1022,Seniorenheim,retirement home
...,...,...,...,...
227,Pflanzenschauhaus,3273,Pflanzenschauhaus,plant show house
228,Gebäude für andere Erholungseinrichtung,3280,Gebäude für andere Erholungseinrichtung,Building for other recreation facility
229,Schutzhütte,3281,Schutzhütte,shelter
230,Touristisches Informationszentrum,3290,Touristisches Informationszentrum,Tourist information center


In [9]:
lst1 = []
for i in map_dictionary['english_name']:
    lst1.append(('residential' in i)|('Residential' in i)|('Mixed use building with housing' in i)\
               |('farmhouse' in i)|('Forsthaus' in i)|('Ferienhaus'in i)|('weekend house' in i)\
               |('Building for trade and services with housing'in i)\
               |('Building for trade and industry with living' in i))
map_dictionary['residential'] = lst1

In [10]:
map_dictionary

,type,code,german_name,english_name,residential
0,Wohngebäude,1000,Wohngebäude,residential building,True
1,Wohnhaus,1010,Wohnhaus,residential house,True
2,Wohnheim,1020,Wohnheim,dorm,False
3,Kinderheim,1021,Kinderheim,children's home,False
4,Seniorenheim,1022,Seniorenheim,retirement home,False
...,...,...,...,...,...
227,Pflanzenschauhaus,3273,Pflanzenschauhaus,plant show house,False
228,Gebäude für andere Erholungseinrichtung,3280,Gebäude für andere Erholungseinrichtung,Building for other recreation facility,False
229,Schutzhütte,3281,Schutzhütte,shelter,False
230,Touristisches Informationszentrum,3290,Touristisches Informationszentrum,Tourist information center,False


# go through all German attribute files and analyse the distributions in the different regions: 

In [11]:
# extract string

def typecode(x):
    
    lst1 = []
    for i in x:
        a = i.split('.')[0][-4:]
        lst1.append(a)
    return(lst1)



In [12]:
path_directory = r"D:\Nicola project\Nicola project\mapping\jiawei"

for i in os.listdir(path_directory):   # os.listdir(path_directory)
        domain = os.path.abspath(path_directory) 
        region = i[:-4]
        print('region is {}'.format(region) )
        stat= os.path.join(domain,i) 
        if stat.endswith(".csv"):
            data = pd.read_csv(stat)
            
            not_have_type = data['type_source'].isna().sum()  # Count the number of nan values
            not_have_type = data['type_source'].isna().sum()  # Count the number of nan values
            have_type = len(data) - not_have_type
            ratio_of_have_type = have_type / len(data)
            print('num of have type is {}, num of non have type is {}, ratio of have type is {} '.format(have_type,not_have_type,ratio_of_have_type))

            data_copy = data.dropna(subset=['type_source']).copy()  #delete all NaN
            data_copy['type_source'] = data_copy['type_source'].astype(str)  #Change float to str
            data_copy['type_source'] =  typecode(data_copy['type_source'])
            
            df = data_copy.merge(map_dictionary,how='left',left_on='type_source' , right_on='code')
            
            num_residential = len(df[df['residential'] == True])
            num_non_residential = (len(df[df['residential'] == False]))
            ratio_of_residential = len(df[df['residential'] == True])/len(df['residential'])
            print('num of residential is {}, num of non-residential is {}, ratio of residential is {} '.format(num_residential,num_non_residential,ratio_of_residential))
            
            a = df['english_name'].value_counts()   # Count the number of type
            print(a)
            print('********************')

            
            




            
            

region is berlin-gov_attrib
num of have type is 416620, num of non have type is 123506, ratio of have type is 0.7713385395259624 
num of residential is 97666, num of non-residential is 28833, ratio of residential is 0.2344246555614229 
Ferienhaus                                  75270
farmhouse                                   20283
shed                                        14211
Sägewerk                                     8438
converter                                    1715
residential and administrative buildings     1625
Kesselhaus                                   1174
greenhouse                                    757
Stall                                         535
Car wash, car wash, car wash                  498
residential and commercial buildings          330
garbage bunker                                296
Toilette                                      184
Energy supply building                        183
factory                                       175
residential an

num of have type is 2066177, num of non have type is 0, ratio of have type is 1.0 
num of residential is 129, num of non-residential is 2066048, ratio of residential is 6.243414770370592e-05 
Cannot be specified based on sources                   1918072
Business or commercial buildings                         96265
Garage                                                   21903
Stall                                                     5826
hothouse, greenhouse                                      3523
General secondary school                                  3263
church                                                    2227
fire Department                                           2107
sports, gym                                               1883
hospital                                                  1778
Museum                                                     930
administration building                                    903
shelter                                             

# mecklenburg-vorpommern-gov_attrib

In [11]:
map_dictionary = pd.read_csv(r"D:\Nicola project\Nicola project\mapping\residential.csv")
map_dictionary = map_dictionary.drop(columns = ['Unnamed: 0'])  

In [12]:
map_dictionary 

,type,code,german_name,english_name,residential
0,Wohngebäude,1000,Wohngebäude,residential building,True
1,Wohnhaus,1010,Wohnhaus,residential house,True
2,Wohnheim,1020,Wohnheim,dorm,False
3,Kinderheim,1021,Kinderheim,children's home,False
4,Seniorenheim,1022,Seniorenheim,retirement home,False
...,...,...,...,...,...
227,Pflanzenschauhaus,3273,Pflanzenschauhaus,plant show house,False
228,Gebäude für andere Erholungseinrichtung,3280,Gebäude für andere Erholungseinrichtung,Building for other recreation facility,False
229,Schutzhütte,3281,Schutzhütte,shelter,False
230,Touristisches Informationszentrum,3290,Touristisches Informationszentrum,Tourist information center,False


In [13]:
data = pd.read_csv(r"D:\Nicola project\Nicola project\mapping\jiawei\mecklenburg-vorpommern-gov_attrib\mecklenburg-vorpommern-gov_attrib.csv")

In [14]:
data

,id,height,type_source,type,age,floors,source_file
0,Building_DEMVAL73000ACsBp,3.379,http://inspire.ec.europa.eu/codelist/CurrentUs...,NaN,NaN,NaN,lod2_33_386_6020_2_inspire
1,Building_DEMVAL73000Asceu,9.300,http://inspire.ec.europa.eu/codelist/CurrentUs...,NaN,NaN,NaN,lod2_33_386_6020_2_inspire
2,Building_DEMVAL73000ACsyv,5.381,http://inspire.ec.europa.eu/codelist/CurrentUs...,NaN,NaN,NaN,lod2_33_386_6020_2_inspire
3,Building_DEMVAL73000Ascez,2.731,http://inspire.ec.europa.eu/codelist/CurrentUs...,NaN,NaN,NaN,lod2_33_386_6020_2_inspire
4,Building_DEMVAL73000ACsBd,9.520,http://inspire.ec.europa.eu/codelist/CurrentUs...,NaN,NaN,NaN,lod2_33_386_6020_2_inspire
...,...,...,...,...,...,...,...
1125165,Building_DEMVAL75000cfdEt,7.209,http://inspire.ec.europa.eu/codelist/CurrentUs...,NaN,NaN,NaN,lod2_33_452_5924_2_inspire
1125166,Building_DEMVAL75000sggBb,5.196,https://registry.gdi-de.org/id/de.____/versorg...,NaN,NaN,NaN,lod2_33_452_5924_2_inspire
1125167,Building_DEMVAL75000sggAE,7.072,http://inspire.ec.europa.eu/codelist/CurrentUs...,NaN,NaN,NaN,lod2_33_452_5924_2_inspire
1125168,Building_DEMVAL75000GcuxA,6.207,http://inspire.ec.europa.eu/codelist/CurrentUs...,NaN,NaN,NaN,lod2_33_452_5924_2_inspire


In [31]:
data_copy = data.dropna(subset=['type_source']).copy()
data_copy

,id,height,type_source,type,age,floors,source_file
0,Building_DEMVAL73000ACsBp,3.379,http://inspire.ec.europa.eu/codelist/CurrentUs...,NaN,NaN,NaN,lod2_33_386_6020_2_inspire
1,Building_DEMVAL73000Asceu,9.300,http://inspire.ec.europa.eu/codelist/CurrentUs...,NaN,NaN,NaN,lod2_33_386_6020_2_inspire
2,Building_DEMVAL73000ACsyv,5.381,http://inspire.ec.europa.eu/codelist/CurrentUs...,NaN,NaN,NaN,lod2_33_386_6020_2_inspire
3,Building_DEMVAL73000Ascez,2.731,http://inspire.ec.europa.eu/codelist/CurrentUs...,NaN,NaN,NaN,lod2_33_386_6020_2_inspire
4,Building_DEMVAL73000ACsBd,9.520,http://inspire.ec.europa.eu/codelist/CurrentUs...,NaN,NaN,NaN,lod2_33_386_6020_2_inspire
...,...,...,...,...,...,...,...
1125165,Building_DEMVAL75000cfdEt,7.209,http://inspire.ec.europa.eu/codelist/CurrentUs...,NaN,NaN,NaN,lod2_33_452_5924_2_inspire
1125166,Building_DEMVAL75000sggBb,5.196,https://registry.gdi-de.org/id/de.____/versorg...,NaN,NaN,NaN,lod2_33_452_5924_2_inspire
1125167,Building_DEMVAL75000sggAE,7.072,http://inspire.ec.europa.eu/codelist/CurrentUs...,NaN,NaN,NaN,lod2_33_452_5924_2_inspire
1125168,Building_DEMVAL75000GcuxA,6.207,http://inspire.ec.europa.eu/codelist/CurrentUs...,NaN,NaN,NaN,lod2_33_452_5924_2_inspire


In [33]:
def typesource(x):
    
    lst2 = []
    for i in x:
        a = i.split('/')[-1]
        lst2.append(a)
    return(lst2)


In [35]:
data_copy['type_source'] = typesource(data_copy['type_source'])

In [36]:
data_copy

,id,height,type_source,type,age,floors,source_file
0,Building_DEMVAL73000ACsBp,3.379,agriculture,NaN,NaN,NaN,lod2_33_386_6020_2_inspire
1,Building_DEMVAL73000Asceu,9.300,residential,NaN,NaN,NaN,lod2_33_386_6020_2_inspire
2,Building_DEMVAL73000ACsyv,5.381,agriculture,NaN,NaN,NaN,lod2_33_386_6020_2_inspire
3,Building_DEMVAL73000Ascez,2.731,agriculture,NaN,NaN,NaN,lod2_33_386_6020_2_inspire
4,Building_DEMVAL73000ACsBd,9.520,residential,NaN,NaN,NaN,lod2_33_386_6020_2_inspire
...,...,...,...,...,...,...,...
1125165,Building_DEMVAL75000cfdEt,7.209,residential,NaN,NaN,NaN,lod2_33_452_5924_2_inspire
1125166,Building_DEMVAL75000sggBb,5.196,versorgungEntsorgung,NaN,NaN,NaN,lod2_33_452_5924_2_inspire
1125167,Building_DEMVAL75000sggAE,7.072,publicServices,NaN,NaN,NaN,lod2_33_452_5924_2_inspire
1125168,Building_DEMVAL75000GcuxA,6.207,agriculture,NaN,NaN,NaN,lod2_33_452_5924_2_inspire


In [37]:
lst3 = []
for i in data_copy['type_source']:
    lst3.append(('residential' in i)|('Residential' in i)|('Mixed use building with housing' in i)\
               |('farmhouse' in i)|('Forsthaus' in i)|('Ferienhaus'in i)|('weekend house' in i)\
               |('Building for trade and services with housing'in i)\
               |('Building for trade and industry with living' in i))
data_copy['residential'] = lst3

In [38]:
data_copy

,id,height,type_source,type,age,floors,source_file,residential
0,Building_DEMVAL73000ACsBp,3.379,agriculture,NaN,NaN,NaN,lod2_33_386_6020_2_inspire,False
1,Building_DEMVAL73000Asceu,9.300,residential,NaN,NaN,NaN,lod2_33_386_6020_2_inspire,True
2,Building_DEMVAL73000ACsyv,5.381,agriculture,NaN,NaN,NaN,lod2_33_386_6020_2_inspire,False
3,Building_DEMVAL73000Ascez,2.731,agriculture,NaN,NaN,NaN,lod2_33_386_6020_2_inspire,False
4,Building_DEMVAL73000ACsBd,9.520,residential,NaN,NaN,NaN,lod2_33_386_6020_2_inspire,True
...,...,...,...,...,...,...,...,...
1125165,Building_DEMVAL75000cfdEt,7.209,residential,NaN,NaN,NaN,lod2_33_452_5924_2_inspire,True
1125166,Building_DEMVAL75000sggBb,5.196,versorgungEntsorgung,NaN,NaN,NaN,lod2_33_452_5924_2_inspire,False
1125167,Building_DEMVAL75000sggAE,7.072,publicServices,NaN,NaN,NaN,lod2_33_452_5924_2_inspire,False
1125168,Building_DEMVAL75000GcuxA,6.207,agriculture,NaN,NaN,NaN,lod2_33_452_5924_2_inspire,False


In [41]:
num_residential = len(data_copy[data_copy['residential'] == True])
num_non_residential = (len(data_copy[data_copy['residential'] == False]))
ratio_of_residential = len(data_copy[data_copy['residential'] == True])/len(data_copy['residential'])
print('num of residential is {}, num of non-residential is {}, ratio of residential is {} '.format(num_residential,num_non_residential,ratio_of_residential))

a = data_copy['type_source'].value_counts()   # Count the number of type
print(a)
print('********************')
           

num of residential is 467235, num of non-residential is 647377, ratio of residential is 0.41919071389864815 
residential                467235
agriculture                234494
commerceAndServices        224789
verkehr                    121365
industrie                   21920
publicServices              16475
versorgungEntsorgung        15128
industrial                   4593
office                       4295
trade                        3732
residenceForCommunities       586
Name: type_source, dtype: int64
********************
